In [1]:
!pip install hmmlearn
!pip install numpy
!pip install sklearn.preprocessing
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 32.5 MB/s eta 0:00:00


In [2]:
import random
from hmmlearn import hmm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

# 한글 형태소 분석기 초기화
okt = Okt()

# 치킨 메뉴 리스트
chicken_menus = ["BBQ", "페리카나", "BHC", "교촌"]

# 확장된 데이터셋 준비: 사용자 입력 문장과 대응하는 상태(의도)
inputs = [
    "안녕", "안녕하세요", "하이", "반가워요", "좋은 하루에요",
    "치킨 추천해줘", "뭐 먹을까?", "오늘 뭐 먹지?", "치킨 추천", "배고파 치킨 먹고 싶어",
    "종료", "그만", "끝내자", "종료하고 싶어", "대화 끝",
    "다시 추천해줘", "또 추천해줘", "더 추천해줘", "다른 치킨 추천해줘", "색다른 메뉴 추천",
    "치킨 종류 알려줘", "어떤 치킨 있어?", "메뉴 보여줘", "선택할 수 있는 치킨?", "추천 가능한 치킨 종류는?",
    "BBQ 먹고 싶어", "BBQ 어때?", "BBQ 맛있어?", "BBQ 치킨", "BBQ 치킨 추천",
    "매운 치킨 추천", "매운 맛으로 추천해줘", "매운 치킨 뭐 있어?", "얼얼한 치킨 추천", "매운 음식 좋아해",
    "가장 인기 있는 치킨은?", "제일 인기 있는 치킨은?", "인기 메뉴 알려줘", "사람들이 많이 먹는 치킨은?", "추천 많이 받는 치킨?",
    "교촌 어때?", "교촌 추천", "교촌이 맛있어?", "교촌 치킨", "교촌 치킨 추천",
    "페리카나가 좋아", "페리카나 추천해줘", "페리카나 치킨", "페리카나 메뉴는?", "페리카나 어때?",
    "BHC 추천해줘", "BHC 치킨 어때?", "BHC 메뉴", "BHC 추천", "BHC가 좋아",
    "양념 치킨 먹고 싶어", "양념 치킨 추천", "달콤한 치킨 추천", "양념이 맛있는 치킨?", "양념으로 추천해줘",
    "후라이드 어때?", "후라이드 추천", "후라이드 맛있어?", "기본 치킨 추천", "오리지널 치킨 추천",
    "치킨 주문", "치킨 주문할게", "주문 도와줘", "치킨 배달", "치킨 시켜줘"
]
states = [
    "GREETING", "GREETING", "GREETING", "GREETING", "GREETING",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "EXIT", "EXIT", "EXIT", "EXIT", "EXIT",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "INQUIRY", "INQUIRY", "INQUIRY", "INQUIRY", "INQUIRY",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "SPICY", "SPICY", "SPICY", "SPICY", "SPICY",
    "POPULAR", "POPULAR", "POPULAR", "POPULAR", "POPULAR",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION", "RECOMMENDATION",
    "SPICY", "SPICY", "SPICY", "SPICY", "SPICY",
    "INQUIRY", "INQUIRY", "INQUIRY", "INQUIRY", "INQUIRY",
    "ORDER", "ORDER", "ORDER", "ORDER", "ORDER"
]

# 한글 전처리 및 토큰화 함수
def preprocess_korean_input(text):
    tokens = okt.morphs(text)
    return ' '.join(tokens)

# 텍스트 전처리를 위한 CountVectorizer 사용 (한글 토큰화 반영)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([preprocess_korean_input(text) for text in inputs])

# 라벨 인코더로 텍스트 데이터를 숫자로 변환
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(states)

# HMM 모델 초기화 및 훈련
model = hmm.MultinomialHMM(n_components=len(set(states)), n_iter=100, tol=0.01)
model.fit(X.toarray().reshape(-1, 1))

def predict_state(user_input):
    """사용자 입력을 받아 상태를 예측합니다."""
    user_input_processed = preprocess_korean_input(user_input)
    input_vector = vectorizer.transform([user_input_processed]).toarray()

    try:
        logprob, state_sequence = model.decode(input_vector.reshape(-1, 1), algorithm="viterbi")
        predicted_state = label_encoder.inverse_transform([state_sequence[0]])[0]
    except ValueError:
        predicted_state = "UNKNOWN"

    return predicted_state

def choice_chicken():
    print("안녕하세요! 저는 치킨 메뉴를 추천해주는 챗봇입니다.")
    name = input("우선, 당신의 이름을 알려주세요: ")
    print(f"반갑습니다, {name}님! 저는 다양한 치킨 메뉴를 추천해드릴 수 있어요.")

    while True:
        user_input = input("\n어떤 치킨을 추천해드릴까요? (종료하려면 '종료'라고 입력): ")

        if user_input.lower() == "종료":
            print("대화를 종료합니다. 맛있게 드세요~")
            break

        # 사용자 입력에 따라 상태 예측
        predicted_state = predict_state(user_input)

        # 예측된 상태에 따라 응답 생성
        if predicted_state == "GREETING":
            print("안녕하세요! 무엇을 도와드릴까요?")
        elif predicted_state == "RECOMMENDATION":
            recommended_menu = random.choice(chicken_menus)
            print(f"{recommended_menu} 치킨을 추천드려요! 맛있게 드세요, {name}님!")
        elif predicted_state == "SPICY":
            print(f"매운 치킨을 원하신다면 {random.choice(['BHC', '교촌'])}의 매운 치킨을 추천드려요!")
        elif predicted_state == "POPULAR":
            print("가장 인기 있는 치킨은 BBQ와 교촌입니다. 어떤 걸 드실래요?")
        elif predicted_state == "INQUIRY":
            print(f"치킨 종류는 {', '.join(chicken_menus)}가 있습니다. 무엇을 드시고 싶으신가요?")
        elif predicted_state == "ORDER":
            print("주문을 도와드리겠습니다! 어떤 치킨을 원하시나요?")
        else:
            print("죄송해요, 잘 이해하지 못했어요. 다시 말씀해주시겠어요?")

# Choice_Chicken 챗봇 시작
choice_chicken()

https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


안녕하세요! 저는 치킨 메뉴를 추천해주는 챗봇입니다.
우선, 당신의 이름을 알려주세요: 홍길동
반갑습니다, 홍길동님! 저는 다양한 치킨 메뉴를 추천해드릴 수 있어요.

어떤 치킨을 추천해드릴까요? (종료하려면 '종료'라고 입력): 매운 치킨
주문을 도와드리겠습니다! 어떤 치킨을 원하시나요?

어떤 치킨을 추천해드릴까요? (종료하려면 '종료'라고 입력): 매운거
주문을 도와드리겠습니다! 어떤 치킨을 원하시나요?

어떤 치킨을 추천해드릴까요? (종료하려면 '종료'라고 입력): 종료
대화를 종료합니다. 맛있게 드세요~
